## Setup for TBMaLT examples

In [10]:
import os
from os.path import join
import urllib, tempfile, tarfile, zipfile

import torch

from tbmalt.io.skf import Skf
torch.set_default_dtype(torch.float64)

## Download auorg-1-1 parameter set

In [11]:
# Link to the auorg-1-1 parameter set
link = 'https://github.com/dftbparams/auorg/releases/download/v1.1.0/auorg-1-1.tar.xz'

# Elements of interest
elements = ['H', 'C', 'N', 'O', 'S', 'Au']

# Path of the parameter set
try:
    os.mkdir("./data")
except FileExistsError:
    pass
    
parameter_db_path = "./data/example_dftb_parameters.h5"

with tempfile.TemporaryDirectory() as tmpdir:

    # Download and extract the auorg parameter set to the temporary directory
    urllib.request.urlretrieve(link, path := join(tmpdir, 'auorg-1-1.tar.xz'))
    with tarfile.open(path) as tar:
        tar.extractall(tmpdir)

    # Select the relevant skf files and place them into an HDF5 database
    skf_files = [join(tmpdir, 'auorg-1-1', f'{i}-{j}.skf')
                 for i in elements for j in elements]

    for skf_file in skf_files:
        Skf.read(skf_file).write(parameter_db_path)

## Download data for dos training

In [12]:
# Link to the dos training data
link = 'https://zenodo.org/records/14677787/files/tbmalt_data.zip?download=1'

# Path of the data
output_file = "./data/dos.zip"

# Download the data
req = urllib.request.Request(link)
with urllib.request.urlopen(req) as response, open(output_file, 'wb') as out_file:
    out_file.write(response.read())

# Extract the data
with zipfile.ZipFile(output_file, 'r') as zip_ref:
    zip_ref.extractall("./data/")

os.remove(output_file)